----

**CRUCES ENTRE SABER11 y SABER PRO**

Saber Pro contienen un identificador único por estudiante (“estu_consecutivo”)
para cada uno de los exámenes que, luego de aplicar un algoritmo de
emparejamiento, permite identificar los estudiantes que presentaron el examen:
1. Saber 11° entre 2010-1 y 2021-2,
2. Saber Pro, entre 2012 y 2023-1
3. Saber TyT entre la primera aplicación de 2016 y 2023-1.

El algoritmo de emparejamiento utiliza seis criterios en los que se hace uso de
llaves creadas a partir de la información de nombres y apellidos completos,
documento de identidad y fecha de nacimiento de los estudiantes. Además, en
caso de que el tipo y número del documento de identidad difieran entre los
exámenes, se utiliza información de la Registraduría Nacional del Estado Civil para
identificar al estudiante en ambos exámenes.

**SOBRE SABER 11**: *Tomamos muestra desde 2012-2*

**Modificacion:**
Con el objetivo de consolidar un Sistema Nacional de Evaluación Estandarizada 
(SNEE) que consiga la alineación de todos los exámenes que lo conforman, la 
estructura del examen Saber 11 fue modificada **a partir del segundo semestre de 2014**
para que sus resultados fueran comparables, en términos de las competencias 
evaluadas, con los de otras pruebas del SNEE como las pruebas Saber 3°, 5° y 9° y el 
examen Saber Pro (Icfes, 2013). Esto llevó a una nueva estructura del examen para 
evaluar las pruebas genéricas: matemáticas, lectura crítica, ciencias naturales, 
sociales y ciudadanas e inglés.



**Periodos recalificados:**

En el marco de la Resolución 503 de 2014 del Icfes, bajo la cual se definieron la 
clasificación de planteles y la estructura del examen Saber 11 vigentes, se planteó la 
necesidad de recalificar los resultados de los estudiantes en algunos de los periodos que 
preceden al cambio en la estructura del examen, ocurrido en el segundo semestre 
de 2014, con el fin de tener resultados comparables y cumplir de esta manera con los 
requerimientos técnicos de la metodología de clasificación vigente. 
Esta metodología requiere de resultados comparables de los estudiantes, en los 
últimos tres años, de cada establecimiento educativo y sus sedes. Debido a la adopción 
de la nueva estructura del examen Saber 11 y con el fin de presentar los resultados de 
la clasificación de planteles vigente desde el segundo semestre del 2014, **se realizó un 
proceso de recalificación de los puntajes de los estudiantes que tomaron el examen en 
el segundo semestre de 2012 y 2013 con el fin de hacerlos comparables a la 
estructura vigente y contar con los tres años de resultados para cada plantel 
educativo**

Con el fin de publicar los resultados de la clasificación de planteles cada semestre, 
como quedó establecido en la Resolución 503 de 2014, la recalificación de los puntajes 
de los estudiantes se hizo también para el **primer semestre del 2013 y 2014** para 
calcular la clasificación de planteles del primer semestre de 2015

----

**SOBRE SABER PRO**: *Tomamos muestra del 2016 en adelante*

Los estudiantes que actualmente toman el examen de Saber Pro son evaluados en 
competencias específicas de acuerdo al grupo de referencia al que pertenece el programa y de 
acuerdo a lo que consideren los directores de programas cuando escogen la combinatoria de 
módulos más acordes a su malla curricular. A pesar de que se evalúan las mismas 
competencias específicas que en el periodo anterior, **a partir del 2016 se produjeron cambios 
en el diseño de los módulos y en la calificación de las mismas que imposibilitan la 
comparación entre años del periodo anterior con el actual**. Los resultados de los módulos de 
competencias específicas para este periodo resultaron en una nueva escala con año base en 
2016.


-------

**DATOS UTILIZADOS POR EL ICFES PARA EL CÁLCULO DEL VALOR AGREGADO**

- Saber Pro:
    - Competencias genéricas

- Saber 11
- NBC




-----

### Constantes y Librerias 

In [5]:
import os
import glob

import pandas as pd
#

import constants as constants
from unidecode import unidecode

import csv
pd.set_option('display.float_format', '{:,.0f}'.format)


### Funciones

In [4]:
def resumen_nans_df(df):
    """
    Devuelve un DataFrame con el número de NaNs, número de no-NaNs y el porcentaje de NaNs por columna,
    ordenado de mayor a menor porcentaje de NaNs.
    """
    total_filas = df.shape[0]
    nans_por_columna = df.isna().sum()
    no_nans = total_filas - nans_por_columna
    porcentaje_nans = (nans_por_columna / total_filas) * 100

    resumen = pd.DataFrame({
        'n_nans': nans_por_columna.astype(float),
        'n_no_nans': no_nans.astype(float),
        'porcentaje_nans': porcentaje_nans.astype(float)
    })

    resumen = resumen.sort_values(by='porcentaje_nans', ascending=False)
    return resumen

In [3]:
#Codigo DANE de Bogota
COD_MCIPIO_BOGOTA = 11001

In [4]:
#Municipios 
BOGOTA_REGION_NOMBRES = [
    "bogota_dc",
    "arbelaez",
    "cabrera",
    "cajica",
    "carmen_de_carupa",
    "caqueza",
    "chia",
    "chipaque",
    "choachi",
    "choconta",
    "cogua",
    "cota",
    "cucunuba",
    "fusagasuga",
    "fomeque",
    "fosca",
    "fuquene",
    "gachala",
    "gachancipa",
    "gacheta",
    "gama",
    "granada",
    "guacheta",
    "guatavita",
    "guasca",
    "gutierrez",
    "junin",
    "la_calera",
    "lenguazaque",
    "macheta",
    "manta",
    "medina",
    "nemocon",
    "pandi",
    "pasca",
    "quetame",
    "san_bernardo",
    "sesquile",
    "sibate",
    "silvania",
    "simijaca",
    "soacha",
    "sopo",
    "suesca",
    "susa",
    "sutatausa",
    "tabio",
    "tausa",
    "tenjo",
    "tibacuy",
    "tibirita",
    "tocancipa",
    "ubala",
    "ubate",
    "ubaque",
    "une",
    "venecia",
    "villapinzon",
    "zipaquira"
]

### Lectura codigo DANE municipios

In [5]:
municipios = pd.read_excel("../../data/Municipios_raw/codigos_municipios.xlsx")

In [6]:
#Renombrar columnas
municipios = municipios.rename(columns={
    'Descripcion': 'nombre_municipio',
    'CodigoDane': 'codigo_dane_municipio'
})

In [12]:
# Filtrar las filas donde los primeros 2 dígitos de 'CodigoDane' sean '25' y '11'
#25 representa el codigo de cundinamarca y 11 de bogotá
municipios = municipios[municipios['codigo_dane_municipio'].astype(str).str[:2].isin(['25','11'])]
#estandarizar nombres de los municipios
municipios['nombre_municipio'] = municipios['nombre_municipio'].apply(
    lambda x: unidecode(x).lower().replace(',', '').replace('.', '').replace(' ', '_')
)
#Escoger los municipios de Bogotá Región
municipios = municipios[municipios['nombre_municipio'].isin(BOGOTA_REGION_NOMBRES)]

In [13]:
print(f"Municipios en lista {len(BOGOTA_REGION_NOMBRES)}")
print(f"Municipios en DF {len(municipios)}")

Municipios en lista 59
Municipios en DF 58


In [14]:
#mirar municipios restantes
descripcion_set = set(municipios['nombre_municipio'])
bogota_region_set = set(BOGOTA_REGION_NOMBRES)

# Obtener diferencia: elementos en municipios pero no en BOGOTA_REGION_NOMBRES
diferencia_1 = descripcion_set - bogota_region_set
diferencia_2 = bogota_region_set - descripcion_set
print("Municipios no cruzaron")
[diferencia_1, diferencia_2]

Municipios no cruzaron


[set(), {'ubate'}]

In [15]:
#agregar ubate: https://www.saludcapital.gov.co/Biblioteca%20de%20Documentos%20DPS%20RIPS/Codificaciones/Codificaci%C3%B3n%20de%20Municipios%20por%20Departamento.pdf
#crear la fila correspondiente
nueva_fila = pd.DataFrame([{'codigo_dane_municipio': 25843, 'nombre_municipio': 'ubate'}])
#agregar al df
municipios = pd.concat([municipios, nueva_fila], ignore_index=True)

In [16]:
municipios.head()

,codigo_dane_municipio,nombre_municipio
0,11001,bogota_dc
1,25053,arbelaez
2,25120,cabrera
3,25126,cajica
4,25151,caqueza


In [17]:
#Se guarda el df con los codigos y nombres de los municipios de bogota region
municipios.to_csv("../../data/Municipios_cleaned/municipios.csv", index=False)

### Lectura de las llaves para cruzar el saber pro con el saber 11

In [18]:
#csv con las llaves para cruzar el saber pro con saber 11
llaves = pd.read_csv("../../data/LLAVES/Llave_Saber11_SaberPro.txt", sep=";")
llaves.head()

,estu_consecutivo_sbpro,estu_consecutivo_sb11
0,EK201210000628,SB11201020119179
1,EK201210001522,SB11201020119180
2,EK201210001647,SB11201020110588
3,EK201210001663,SB11201020116060
4,EK201210003701,SB11201020120759


### Procesamiento informacion cruda del saber 11

In [19]:
# Asume que el notebook está en /src
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, "../.."))

# Rutas absolutas a los directorios de datos
data_dir_sb11_before20142 = os.path.join(project_root, "data", "SABER11_raw", "ANTES20142")
data_dir_sb11_after20142 = os.path.join(project_root, "data", "SABER11_raw", "DESPUES20142")

# Buscar archivos
csv_files_before20142 = glob.glob(os.path.join(data_dir_sb11_before20142, "*.txt"))
csv_files_after20142 = glob.glob(os.path.join(data_dir_sb11_after20142, "*.txt"))

In [20]:
#Variables a seleccionar del saber 11 antes del 2014-2 
cols_before_20142 = constants.cols_sb11before_20142
#Variables a seleccionar del saber 11 despues del 2014-2 (inclusive) 
cols_after_20142 = constants.cols_sb11after_20142

In [21]:
import pandas as pd

# Lista para guardar los dataframes
data_list = []
descartados = []  # Archivos descartados por estar vacíos o completamente NA

# Función para leer con columnas opcionales
def read_csv_with_optional_columns(file_path, required_cols):
    try:
        # Leer solo el header
        with open(file_path, 'r', encoding='utf-8') as f:
            header_line = f.readline()
        available_cols = header_line.strip().split(";")

        # Determinar columnas que sí existen en el archivo
        existing_cols = [col for col in required_cols if col in available_cols]
        missing_cols = [col for col in required_cols if col not in available_cols]

        # Leer solo las columnas disponibles
        df = pd.read_csv(file_path, sep=";", usecols=existing_cols, low_memory=False)

        # Añadir las columnas faltantes como vacías
        for col in missing_cols:
            df[col] = pd.NA

        # Reordenar las columnas según el orden original
        df = df[required_cols]
        return df

    except Exception as err:
        print(f"Error reading {file_path}: {err}")
        return None


# Procesar archivos antes de 2014-2
for file_path in csv_files_before20142:
    df = read_csv_with_optional_columns(file_path, cols_before_20142)
    if df is not None:
        if df.empty or df.isna().all(axis=1).all():
            descartados.append(file_path)
        else:
            data_list.append(df)

# Procesar archivos después de 2014-2
for file_path in csv_files_after20142:
    df = read_csv_with_optional_columns(file_path, cols_after_20142)
    if df is not None:
        if df.empty or df.isna().all(axis=1).all():
            descartados.append(file_path)
        else:
            data_list.append(df)

# Log de archivos descartados
print(f"\n{len(descartados)} archivo(s) fueron descartados por estar vacíos o sin datos útiles:")
for file in descartados:
    print(f" - {file}")

# Concatenar dataframes restantes
sb11 = pd.concat(data_list, ignore_index=True)
print("ya paso el concat")

sb11["cole_cod_mcpio_ubicacion"] = sb11["cole_cod_mcpio_ubicacion"].astype("Int64")

print("ya paso el astype")

# Ordenar por periodo y resetear índices
#sb11 = sb11.sort_values(by="periodo").reset_index(drop=True)

#print("Ya paso el sort")

# Liberar memoria
del data_list
del df
#del descartados


0 archivo(s) fueron descartados por estar vacíos o sin datos útiles:


C:\Users\user\AppData\Local\Temp\ipykernel_21496\4054654724.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sb11 = pd.concat(data_list, ignore_index=True)


ya paso el concat
ya paso el astype


In [22]:
#Añadir a sb11 las 2 llaves para cruzar el saber 11 con el saber pro
#estu_consecutivo_sb11: llave del saber 11
#estu_consecutivo_sbpro:  llave del saber pro
base_sb11 = pd.merge(
    sb11,
    llaves,
    how = "left", #por completitud de la informacion usamos left merge
    left_on = "estu_consecutivo",
    right_on = "estu_consecutivo_sb11")

base_sb11.rename(columns={"estu_consecutivo_sbpro":"llave_saber_pro"},inplace=True) 
base_sb11.rename(columns={"estu_consecutivo_sb11":"llave_saber_11"},inplace=True) 

del sb11
del llaves

In [23]:
#reorganizamos las columnas
base_sb11 = base_sb11[constants.cols_sb11]

#parse las columnas a los tipos de variables adecuados
for col, dtype in constants.dtype_mapping_saber11.items():
    if col in base_sb11.columns:
        try:
            base_sb11[col] = base_sb11[col].astype(dtype)
        except Exception as e:
            print(f"No se pudo convertir la columna {col} a {dtype}: {e}")

In [24]:
base_sb11.periodo.unique()

array([20122, 20131, 20132, 20141, 20142, 20151, 20152, 20161, 20162,
       20171, 20172, 20181, 20182, 20191, 20192, 20201, 20202, 20211,
       20212, 20221, 20222, 20231, 20232, 20241, 20242])

In [25]:
#Guardar el dataframe como un archivo csv en la carpeta SABER11_cleaned
base_sb11.to_csv("../../data/SABER11_cleaned/base_sb11.csv", index=False)
#del base_sb11

Mirar la cantidad de nans por columna para todas las observaciones del ICFES

-----

In [7]:
sb11 = pd.read_csv("../../data/SABER11_cleaned/base_sb11.csv", sep=",")

C:\Users\user\AppData\Local\Temp\ipykernel_6056\3119327766.py:1: DtypeWarning: Columns (1,2,7,24,25,29,30,31,33,35,36,37,38,39,40,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  sb11 = pd.read_csv("../../data/SABER11_cleaned/base_sb11.csv", sep=",")


In [10]:
resumen_nans_base_sb11 = resumen_nans_df(sb11)

filtro = sb11['llave_saber_11'].notna() & sb11['llave_saber_pro'].notna()
base_filtrado = sb11[filtro]

resumen_nans_base_sb11_filtrado = resumen_nans_df(base_filtrado)


----

### Saber PRO

In [ ]:
# Ruta del directorio con los archivos
data_dir_saberpro = "../../data/SABERPRO_raw_reduced/"
csv_files_saberpro = glob.glob(os.path.join(data_dir_saberpro, "*.txt"))
# Normaliza la ruta antes de remover
archivo_a_remover = os.path.normpath('../../data/SABERPRO_raw_reduced/saberpro_20232.txt')
csv_files_saberpro = [os.path.normpath(f) for f in csv_files_saberpro]

if archivo_a_remover in csv_files_saberpro:
    csv_files_saberpro.remove(archivo_a_remover)
else:
    print("El archivo a remover no está en la lista.")

In [14]:
#lista para guardar los dataframes
data_list = []

#Por separado leer el archivo del saber pro del 2023-2 porque el separador es distinto
data_temp = pd.read_csv(
    f"{data_dir_saberpro}saberpro_20232.txt", 
    encoding="utf-8",
    sep = ";",
    usecols=constants.cols_saberpro_lower
)

data_list.append(data_temp)

##Leer los archivos del saber pro antes del 2023-2
for file_path in csv_files_saberpro:
    try:
        data_temp = pd.read_csv(
            file_path,
            sep="¬",
            usecols=constants.cols_saberpro_upper,
            engine="python",
            encoding="utf-8",
            quoting=csv.QUOTE_NONE
        )
        #Castear columna a tipo int
        data_temp["ESTU_COD_MCPIO_PRESENTACION"] = data_temp["ESTU_COD_MCPIO_PRESENTACION"].astype("Int64")
        #pasar los nombres de las columnas a minuscula
        data_temp.columns = data_temp.columns.str.lower()
        data_list.append(data_temp)
        
    except Exception as err:
        print(err)
        print(f"Error reading {file_path}: {err}")
        

#concatenar los dataframes
sbpro = pd.concat(data_list, ignore_index=True)

#Quedarse con las observaciones del saber pro donde el programa educativo esta ubicado en Bogota 
#sbpro = sbpro[sbpro["estu_prgm_codmunicipio"] == COD_MCIPIO_BOGOTA]

#Quedarse con las observaciones del saber pro donde el programa educativo esta ubicado en Bogota-Region 
#El 18 de julio se decidio quitar el filtro por municipio
#sbpro = sbpro[sbpro["estu_prgm_codmunicipio"].isin(municipios["codigo_dane_municipio"])]

sbpro = sbpro.sort_values(by="periodo")
sbpro = sbpro.reset_index(drop=True)

#Reordenar columnas
sbpro = sbpro[constants.cols_saberpro_lower]

del data_list
del data_temp

In [15]:
#parse las columnas a los tipos de variables adecuados
for col, dtype in constants.dtype_mapping_saberpro.items():
    if col in sbpro.columns:
        try:
            sbpro[col] = sbpro[col].astype(dtype)
        except Exception as e:
            print(f"No se pudo convertir la columna {col} a {dtype}: {e}")

In [9]:
resumen_nans_sbpro = resumen_nans_df(sbpro)

In [21]:
#Guardar el dataframe como un archivo csv en la carpeta SABERPRO_cleaned
sbpro.to_csv("../../data/SABERPRO_cleaned/base_sbpro.csv", index=False)

### Cargar las bases limpias del saber 11 y saber pro

In [83]:
num_duplicates = sbpro['estu_consecutivo'].duplicated().sum()
num_obs = sbpro.periodo.count()
print("----SABER PRO----")
print("Numero Observaciones:")
print(num_obs)

print("Observaciones duplicadas:")
print(num_duplicates)

print("Observaciones unicas:")
print(num_obs-num_duplicates)


num_duplicates = sb11['estu_consecutivo'].duplicated().sum()
num_obs = sb11.periodo.count()
print("----SABER 11----")
print("Numero Observaciones:")
print(num_obs)

print("Observaciones duplicadas:")
print(num_duplicates)

print("Observaciones unicas:")
print(num_obs-num_duplicates)

----SABER PRO----
Numero Observaciones:
812065
Observaciones duplicadas:
1893
Observaciones unicas:
810172
----SABER 11----
Numero Observaciones:
8562110
Observaciones duplicadas:
0
Observaciones unicas:
8562110


In [ ]:
#sbpro = sbpro.drop_duplicates("estu_consecutivo",keep="first")
##TODO: Analizar las observaciones duplicadas** (se eliminó el 0.2% de la info del sbpro )

------

### MERGE SABER PRO con SABER 11

In [2]:
import pandas as pd

In [3]:
sb11 = pd.read_csv("../../data/SABER11_cleaned/base_sb11.csv", sep=",")
sbpro = pd.read_csv("../../data/SABERPRO_cleaned/base_sbpro.csv", sep=",")

C:\Users\user\AppData\Local\Temp\ipykernel_8664\3907161097.py:1: DtypeWarning: Columns (1,2,7,24,25,29,30,31,33,35,36,37,38,39,40,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  sb11 = pd.read_csv("../../data/SABER11_cleaned/base_sb11.csv", sep=",")


In [6]:
#Unimos la base consolidada del Saber 11 con la base consolidada del Saber PRO
#Right join
data = pd.merge(sb11,sbpro, how="right", left_on = "llave_saber_pro", right_on = "estu_consecutivo", suffixes= ("_bdsaber11","_bdsaberpro"))

In [7]:
resumen_nans_muestra_saber11_saberpro = resumen_nans_df(data)

In [11]:
with pd.ExcelWriter('../../data/Observaciones_perdidas/resumen_nans.xlsx') as writer:
    resumen_nans_base_sb11.to_excel(writer, sheet_name='Resumen_Saber11')
    resumen_nans_base_sb11_filtrado.to_excel(writer, sheet_name='Resumen_Saber11_Filtrado')
    resumen_nans_sbpro.to_excel(writer, sheet_name='Resumen_SaberPro')
    resumen_nans_muestra_saber11_saberpro.to_excel(writer, sheet_name='Resumen_Saber11_SaberPro')

### Descripcion Base de Datos consolidada

Características de esta base:
- Existen estudiantes en la base del Saber 11 que no tienen llave para unirse con la base del Saber Pro. 
    - Recordar las llaves: estu_consecutivo_sbpro y estu_consecutivo_sb11
- No existen estudiantes duplicados (var: estu_consecutivo_icfes) en la base del saber 11
- Existen duplicados (var: estu_consecutivo_pro) en la base del saber pro

In [12]:
print(data.shape)
data.head()

(1948415, 68)


,estu_consecutivo_bdsaber11,llave_saber_11,llave_saber_pro,periodo_bdsaber11,punt_global_bdsaber11,cole_cod_depto_ubicacion,cole_cod_mcpio_ubicacion,cole_depto_ubicacion,recaf_punt_sociales_ciudadanas,recaf_punt_ingles,...,inst_caracter_academico,estu_inst_codmunicipio,estu_inst_municipio,estu_inst_departamento,estu_cod_mcpio_presentacion,mod_razona_cuantitat_punt,mod_lectura_critica_punt,mod_competen_ciudada_punt,mod_ingles_punt,mod_comuni_escrita_punt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,UNIVERSIDAD,50001,VILLAVICENCIO,META,NaN,183,132,126,146,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,UNIVERSIDAD,8001,BARRANQUILLA,ATLANTICO,NaN,182,197,195,217,NaN
2,SB11201220353778,SB11201220353778,EK201620000561,"20,122",NaN,11,"11,001",BOGOTA,49,54,...,INSTITUCIÓN UNIVERSITARIA,11001,BOGOTÁ D.C.,BOGOTA,NaN,163,184,165,160,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,UNIVERSIDAD,76001,CALI,VALLE,NaN,139,186,185,223,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,UNIVERSIDAD,25175,CHÍA,CUNDINAMARCA,NaN,117,175,149,213,NaN


**Calcular Nuevas columnas** 
- Diferencia entre el periodo del Saber Pro y el Saber11
    - Para el cálculo del VA, el ICFES utiliza una diferencia de 4-8 años entre la evaluación del Saber 11 y el Saber Pro para los programas distintos a medicina. Medicina tiene un rango de 4-9 años
     

In [13]:
data["dif_periodos"] = data["periodo_bdsaberpro"]- data["periodo_bdsaber11"]
data['anio_presentacion_sbpro'] = data['periodo_bdsaberpro']//10
data['anio_presentacion_sb11'] = data['periodo_bdsaber11']//10

In [14]:
data[['periodo_bdsaberpro',"periodo_bdsaber11","dif_periodos",'anio_presentacion_sbpro','anio_presentacion_sb11']].sample(5)

,periodo_bdsaberpro,periodo_bdsaber11,dif_periodos,anio_presentacion_sbpro,anio_presentacion_sb11
90344,20163,NaN,NaN,2016,NaN
479808,20173,NaN,NaN,2017,NaN
1752045,20231,NaN,NaN,2023,NaN
1685771,20225,"20,172",53,2022,"2,017"
1325028,20212,NaN,NaN,2021,NaN


In [15]:
data = data.sort_values(by="periodo_bdsaber11").reset_index(drop=True)

In [16]:
#Guardamos la base de datos consolidada en el subdirectorio BD
data.to_csv("../../data/BD/saber11_nacional_saberpro_bogota_region.csv",index=False)
data.to_pickle("../../data/BD/saber11_nacional_saberpro_bogota_region.pkl")

In [31]:
sb11.to_pickle("../../data/SABER11_cleaned/base_sb11.pkl")
sbpro.to_pickle("../../data/SABERPRO_cleaned/base_sbpro.pkl")